<a href="https://colab.research.google.com/github/anaguilarar/agwise_data_sourcing/blob/main/GEEMODIS_data_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ag-Wise Data Sourcing Tutorial

## MODIS/VIIRS Vegetation Index Time Series Downloader

This tutorial explains how to use this notebook to download, process, and visualize vegetation index (VI) time series (e.g., NDVI) from MODIS or VIIRS using Google Earth Engine (GEE).

Firs you need to clone the repository

In [ ]:
import os

if not os.path.exists('agwise_data_sourcing'):
  !git clone https://github.com/anaguilarar/agwise_data_sourcing.git
  os.chdir('/content/agwise_data_sourcing')
else:
  os.chdir('/content/agwise_data_sourcing')

## Workflow Overview
1. **Country Example Configuration** – Select area, time, and satellite product.
2. **Data Downloading** – Retrieve NDVI time series using GEE.
3. **Data Preprocessing** – Fill gaps and smooth temporal noise.
4. **Crop Masking** – Focus analysis on cropland areas.
5. **Visualization & Interpretation** – Inspect NDVI patterns and growth trends.

### Country Example Configuration


In this section, you will set the parameters for your analysis. Modify the dictionary below to match your region and product of interest.


- `ADM0_NAME` define the administrative levels.
- `product` sets the MODIS/VIIRS dataset.
- `starting_date` and `ending_date` specify the period of interest.


Example: NDVI extraction for **Kenya – Coast Province (2023)**

In [ ]:
### INITIAL configuration

configuration = {
    'GENERAL_SETTINGS':{
      'ee_project_name': 'ee-anaguilarar'

      },
    'PREPROCESSING':{
        'crop_mask': True,
        'crop_mask_product': 'ESA'
    },
    'DATA_DOWNLOAD':
     {
      'ADM0_NAME': 'Kenya',
      'ADM1_NAME': 'Coast',
      'ADM2_NAME': None,
      'product': 'MOD13Q1',
      'starting_date': '2023-01-01',
      'ending_date': '2023-12-01',

    }
}


- The first time you run this notebook, GEE will request authentication (`ee.Authenticate()`).
- Depending on your area size, the data request might take a few minutes.

In [ ]:
from gee_datasets.gee_data import GEEMODIS
import ee
ee.Initialize(project=configuration['GENERAL_SETTINGS']['ee_project_name'])


## Data Downloading


This section connects to Google Earth Engine, defines your region of interest, and retrieves the vegetation index time series.


Steps:
1. **Initialize Google Earth Engine (GEE)** with your project.
2. **Create the downloader object** (`GEEMODIS`).
3. **Run the query** to retrieve the imagery.

In [ ]:
data_downloader = GEEMODIS(configuration['DATA_DOWNLOAD']['ADM0_NAME'], configuration['DATA_DOWNLOAD']['product'])
data_downloader.initialize_query(configuration['DATA_DOWNLOAD']['starting_date'], configuration['DATA_DOWNLOAD']['ending_date'])


In [ ]:
data_downloader.plot_time_series('NDVI')

### Data Preprocessing


Once you have the raw NDVI time series, you can apply two important steps to make it analysis-ready:


1. **Gap Filling**: Handles missing data due to cloud cover or sensor issues using linear interpolation.
2. **Smoothing**: Applies the Savitzky–Golay filter to reduce noise while preserving vegetation dynamics.



In [ ]:
from gee_datasets.processing_funs import fill_gaps_linear, smooth_ts_using_savitsky_golay_modis, summarize_collection_tots
import matplotlib.pyplot as plt

data_filled = fill_gaps_linear(data_downloader.query, 'NDVI')

print("First smoothed image properties:",
      ee.Image(data_filled.first()).getInfo()["properties"])



In [ ]:
band = 'NDVI'
result = smooth_ts_using_savitsky_golay_modis(data_filled.select('NDVI'), window_size = 5)
print("First smoothed image properties:",
      ee.Image(result.first()).getInfo()["properties"])


In [ ]:
from utils.plots import plot_time_series_from_ic

band2 = 'NDVI_smooth'

df = plot_time_series_from_ic(
    ic=ee.ImageCollection(result),
    band=band2,
    summarize_fn=summarize_collection_tots,
    region_filter=data_downloader.country_filter,
    title=f"{band2} smoothed time series"
)


## Download data for a specific administrative level

You can target data at different administrative levels using the configuration keys:

1. Set `ADM0_NAME` for the country (required).
2. Set `ADM1_NAME` for the first-level admin (province/state) if you want a subregion.
3. Set `ADM2_NAME` for the district/municipality if available and needed.


**Example configuration (Kenya, Coast province):**


```python
configuration['DATA_DOWNLOAD'].update({
'ADM0_NAME': 'Kenya',
'ADM1_NAME': 'Coast',
'ADM2_NAME': None,

})


In [ ]:
data_downloader = GEEMODIS(configuration['DATA_DOWNLOAD']['ADM0_NAME'], configuration['DATA_DOWNLOAD']['product'])
data_downloader.initialize_query(configuration['DATA_DOWNLOAD']['starting_date'], configuration['DATA_DOWNLOAD']['ending_date'])

feature_name = 'Kericho'

not_smootheddf = data_downloader.get_adm_timeseries(adm_level='ADM1', feature_name= feature_name, fill_gaps=True, sg = False)
smootheddf3 = data_downloader.get_adm_timeseries(adm_level='ADM1', feature_name= feature_name, fill_gaps=True, sg = True, window_size = 3)
smootheddf5 = data_downloader.get_adm_timeseries(adm_level='ADM1', feature_name= feature_name, fill_gaps=True, sg = True, window_size = 5)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(not_smootheddf['date'], not_smootheddf[not_smootheddf.columns[1]], marker='o', linestyle='-', label = 'Raw')
plt.plot(smootheddf3['date'], smootheddf3[smootheddf3.columns[1]], marker='o', linestyle='-', c = 'red', label = 'Smoothed window 3')
plt.plot(smootheddf5['date'], smootheddf5[smootheddf5.columns[1]], marker='o', linestyle='-', c = 'green', label = 'Smoothed window 5')

plt.title(f"{not_smootheddf.columns[1]} Time Series for {feature_name}")
plt.xlabel("Date")
plt.ylabel(not_smootheddf.columns[1])
plt.grid(True)
plt.legend()
plt.show()

## Using a Crop Mask


Crop masking helps you focus only on agricultural areas, avoiding urban or forest regions.


You can enable it in the configuration dictionary:
```python
'PREPROCESSING': {
'crop_mask': True,
'crop_mask_product': 'ESA' # or 'DYNAMICWORLD'
}
```


- **ESA WorldCover (ESA)** provides global land cover classes.
- **Dynamic World (Google)** offers near real-time land classification.


When enabled, the notebook filters the NDVI collection to cropland pixels only.

In [ ]:
from gee_datasets.gee_data import GEECropMask, GEEMODIS


data_downloader = GEEMODIS(configuration['DATA_DOWNLOAD']['ADM0_NAME'], configuration['DATA_DOWNLOAD']['product'])
data_downloader.initialize_query(configuration['DATA_DOWNLOAD']['starting_date'], configuration['DATA_DOWNLOAD']['ending_date'])


cropmask_downloader = GEECropMask(configuration['DATA_DOWNLOAD']['ADM0_NAME'], configuration['PREPROCESSING']['crop_mask_product'])
cropmask_downloader.initialize_query()

crop_mask = ee.Image(cropmask_downloader.query.first()).clip(cropmask_downloader.country_filter).eq(cropmask_downloader.crop_mask_value[cropmask_downloader.product])



In [ ]:
data_downloader.country_filter

dataset = ee.FeatureCollection(data_downloader._global_adiminstrative_data.format(adm_level='ADM1'))
dataset.filter(ee.Filter.eq('shapeName', 'Busia'))


Visualization of raw vs. processed time series helps you confirm the effectiveness of preprocessing.


In [ ]:
from gee_datasets.processing_funs import fill_gaps_linear, smooth_ts_using_savitsky_golay_modis

data_filled = fill_gaps_linear(data_downloader.query, 'NDVI')


datamasked = data_downloader.query.first().clip(cropmask_downloader.country_filter).updateMask(crop_mask)

data_filled_masked = data_filled.first().clip(cropmask_downloader.country_filter).updateMask(crop_mask)

data_filled_SG = smooth_ts_using_savitsky_golay_modis(data_filled.select('NDVI'), window_size = 5)


data_filled_SG_masked = data_filled_SG.first().clip(cropmask_downloader.country_filter).updateMask(crop_mask)

In [ ]:
import geemap


# Create a map
Map = geemap.Map(center=[-1.37, 38.01], zoom=8)  # Centered on Kitui, Kenya (example coords)

# Define visualization parameters
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['white', 'green']  # Adjust based on your mask values
}

vis_NDVIparams = {
    'min': 0,
    'max': 10000,

}

# Add the image layer
Map.addLayer(crop_mask, vis_params, "Masked Crop Image")

Map.addLayer(datamasked.select('NDVI'), vis_NDVIparams, "NDVI Image")
Map.addLayer(data_filled_masked.select('NDVI'), vis_NDVIparams, "NDVI Image filled")
Map.addLayer(data_filled_SG_masked.select('NDVI_smooth'), vis_NDVIparams, "NDVI Image filled + SG")

# Display the map
Map